## Commit

Running the cell below copies any local source code modifications into the FlashAttention install directory on the FlashAttention docker image, and then commits the Docker image.

In [603]:
!./build_commit.sh

"docker kill" requires at least 1 argument.
See 'docker kill --help'.

Usage:  docker kill [OPTIONS] CONTAINER [CONTAINER...]

Kill one or more running containers
Untagged: flash:flattened_live
Deleted: sha256:57d38a542517df19b3f3102b74219f3f240614bc9145517d5fe6b2abab58a5be
Deleted: sha256:260a402d0e978ffb5a66c628f2b2bdc43fc4e399cc367d635fc969a8906f7c59
Creating flash-attention_flashattention_run ... 
ting flash-attention_flashattention_run ... done

torch.__version__  = 1.14.0a0+410ce96


running install
running bdist_egg
running egg_info
writing flash_attn.egg-info/PKG-INFO
writing dependency_links to flash_attn.egg-info/dependency_links.txt
writing requirements to flash_attn.egg-info/requires.txt
writing top-level names to flash_attn.egg-info/top_level.txt
reading manifest file 'flash_attn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
/usr/local/lib/python3.8/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated.

byte-compiling build/bdist.linux-x86_64/egg/flash_attn/flash_attention.py to flash_attention.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gpt.py to gpt.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/llama.py to llama.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/__init__.py to __init__.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gpt_neox.py to gpt_neox.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/gptj.py to gptj.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/bert.py to bert.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/vit.py to vit.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/flash_attn/models/opt.py to opt.cpython-38.pyc
creating stub loader for flash_attn_cuda.cpython-38-x86_64-linux-gnu.so
byte-compiling build/bdist.linux-x86_64/egg/flash_attn_cuda.py t

## Testing

In [ ]:
--capture=tee-sys

In [610]:
!echo "pytest --capture=tee-sys tests/test_flash_attn_lse.py"  | docker-compose run flashattention 

Creating flash-attention_flashattention_run ... 
ting flash-attention_flashattention_run ... done============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.2.0, pluggy-1.0.0
rootdir: /home/workspace
plugins: rerunfailures-10.3, xdoctest-1.0.2, hypothesis-5.35.1, xdist-3.1.0, shard-0.1.2, hydra-core-1.3.1
collected 1061 items
Running 1061 items in this shard

tests/test_flash_attn_lse.py sssssssssssssssssssssssssssssssssssssssssss [  4%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 10%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 17%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 24%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 31%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 37%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 44%]
sssssssssssssss

           -1.4482e+00,  1.0068e+00]]]], device='cuda:0', dtype=torch.float16)
g_output_null: tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],

(tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
         

           -1.8604e+00,  7.3945e+00]]]], device='cuda:0', dtype=torch.float16)
attn_ref tensor([[[[0.0009, 0.0007, 0.0006,  ..., 0.0009, 0.0008, 0.0013],
          [0.0008, 0.0012, 0.0011,  ..., 0.0010, 0.0006, 0.0011],
          [0.0016, 0.0009, 0.0011,  ..., 0.0007, 0.0008, 0.0010],
          ...,
          [0.0016, 0.0007, 0.0005,  ..., 0.0010, 0.0013, 0.0006],
          [0.0007, 0.0009, 0.0008,  ..., 0.0012, 0.0009, 0.0006],
          [0.0007, 0.0011, 0.0009,  ..., 0.0008, 0.0009, 0.0012]],

         [[0.0007, 0.0009, 0.0009,  ..., 0.0011, 0.0008, 0.0010],
          [0.0011, 0.0010, 0.0008,  ..., 0.0014, 0.0014, 0.0011],
          [0.0015, 0.0007, 0.0010,  ..., 0.0008, 0.0008, 0.0012],
          ...,
          [0.0010, 0.0008, 0.0012,  ..., 0.0019, 0.0011, 0.0009],
          [0.0007, 0.0006, 0.0008,  ..., 0.0010, 0.0009, 0.0009],
          [0.0004, 0.0009, 0.0010,  ..., 0.0013, 0.0012, 0.0013]],

         [[0.0012, 0.0008, 0.0010,  ..., 0.0012, 0.0013, 0.0008],
          [0.0008, 0

       device='cuda:0', grad_fn=<SoftmaxBackward0>)
max: tensor(0.0072, device='cuda:0', dtype=torch.float16, grad_fn=<MaxBackward1>)
min: tensor(0.0001, device='cuda:0', dtype=torch.float16, grad_fn=<MinBackward1>)
nan: tensor(False, device='cuda:0')
dq_ref_lse: tensor([[[[-9.1875e+00, -1.7539e+00, -5.3008e+00,  ...,  1.5977e+01,
            9.5703e+00,  9.5000e+00],
          [ 7.8750e+00,  2.3453e+01,  1.0000e+02,  ..., -4.6625e+01,
            4.8188e+01, -9.7734e+00],
          [ 3.2062e+01, -5.7844e+01, -7.4312e+01,  ...,  1.1456e+02,
            3.5562e+01,  1.8766e+01],
          [-1.5305e+01, -1.4102e+01, -1.7422e+01,  ..., -1.2055e+01,
            5.7461e+00,  1.2891e+00]],

         [[ 3.8938e+01, -1.9672e+01, -4.3750e+01,  ..., -1.4262e+02,
           -7.7000e+01, -5.9812e+01],
          [ 8.5000e+00,  4.1953e+00, -5.6719e+00,  ...,  6.0394e-02,
           -6.6719e+00,  4.7656e+00],
          [-2.0801e+00, -1.1426e+00,  3.1016e+00,  ..., -5.6445e+00,
           -3.5762e+00,

           -1.8535e+00,  7.3906e+00]]]], device='cuda:0', dtype=torch.float16)
dq_pt_lse: tensor([[[[-9.1797e+00, -1.7480e+00, -5.3125e+00,  ...,  1.5977e+01,
            9.5625e+00,  9.5000e+00],
          [ 7.8711e+00,  2.3453e+01,  9.9938e+01,  ..., -4.6594e+01,
            4.8156e+01, -9.8047e+00],
          [ 3.2125e+01, -5.7844e+01, -7.4375e+01,  ...,  1.1475e+02,
            3.5594e+01,  1.8734e+01],
          [-1.5273e+01, -1.4062e+01, -1.7391e+01,  ..., -1.2031e+01,
            5.7422e+00,  1.2783e+00]],

         [[ 3.9000e+01, -1.9734e+01, -4.3750e+01,  ..., -1.4288e+02,
           -7.7125e+01, -5.9906e+01],
          [ 8.4922e+00,  4.1953e+00, -5.6758e+00,  ...,  6.1829e-02,
           -6.6719e+00,  4.7617e+00],
          [-2.0781e+00, -1.1406e+00,  3.0977e+00,  ..., -5.6406e+00,
           -3.5723e+00, -3.2324e+00],
          [ 1.7062e+02, -2.5469e+01,  1.8650e+02,  ..., -2.1326e-01,
           -1.2562e+02,  1.0867e+01]],

         [[ 3.6062e+01,  9.2188e+00, -9.1875e+00, 

.sblocksize_c:256
max_seqlen_k:1024
max_seqlen_q:1024
loop:1
Actual dropout fraction: 0.0
Output max diff: 0.0001220703125
Output mean diff: 3.159046173095703e-06
Pytorch max diff: 0.0001220703125
Pytorch mean diff: 3.3974647521972656e-06
Attention max diff: 3.814697265625e-06
Attention Pytorch max diff: 7.62939453125e-06
dout: tensor([[[ 0.8281,  0.8296, -0.5376,  ..., -1.3525,  0.8916,  0.1613],
         [-1.1680,  1.0283, -0.5791,  ...,  0.2285, -1.4805,  0.9668],
         [-0.0906,  0.5923, -0.4556,  ...,  0.4163, -0.5977, -0.9819],
         [-0.0792,  0.7217,  0.0747,  ..., -0.8105,  0.0594,  0.3486]],

        [[ 0.7495, -0.0450, -0.6743,  ..., -1.5117,  1.5010, -0.6401],
         [ 0.5703,  1.3896, -0.0789,  ...,  0.7256, -0.3921,  1.7324],
         [ 0.0973,  0.8550,  2.0664,  ..., -1.9512, -0.0748,  0.3201],
         [ 0.4070, -0.2072,  1.1953,  ..., -0.2323, -0.2008,  0.2029]],

        [[-0.8389, -0.4207, -0.3467,  ...,  1.0322, -0.0865, -0.0842],
         [ 0.1231,  1.7363,

ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 58%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 65%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 71%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 78%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 85%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 92%]
ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss [ 99%]
ssssssssss                                                               [100%]

======================= 2 passed, 1059 skipped in 3.14s ========================
